In [22]:
# python kernal version 3.10
!pip install azure-eventhub==5.11.6 

In [29]:
import toml
import json
from azure.eventhub import EventHubProducerClient, EventData

In [30]:
def send_events_to_event_hub(connection_str, event_hub_name, user_data):
    producer = EventHubProducerClient.from_connection_string(conn_str=connection_str, eventhub_name=event_hub_name)
    try:
        # Ensure user_data is serialized and encoded into bytes
        if isinstance(user_data, dict):  # Assuming user_data is a dictionary
            user_data = json.dumps(user_data).encode('utf-8')
        
        event_batch = producer.create_batch()
        event_batch.add(EventData(user_data))
        producer.send_batch(event_batch)
        print("Event data sent to Event Hub")
    except Exception as e:
        print(f"Failed to send event: {e}")
    finally:
        producer.close()

In [31]:
def read_config_from_toml(file_path):
    """
    Reads the configuration from a TOML file.

    Parameters:
    - file_path: Path to the TOML configuration file.

    Returns:
    A tuple containing the connection string and Event Hub name.
    """
    config = toml.load(file_path)
    return config['event_hub']['connection_string'], config['event_hub']['name']

In [32]:
def load_user_data_from_json(file_path):
    """
    Loads user data from a JSON file.

    Parameters:
    - file_path: Path to the JSON file containing user data.

    Returns:
    A Python data structure containing the user data loaded from the JSON file.
    """
    try:
        with open(file_path, 'r') as file:
            user_data = json.load(file)
            return user_data
    except FileNotFoundError:
        print("The file was not found.")
        return None
    except json.JSONDecodeError:
        print("Error decoding JSON.")
        return None

# Example usage
json_file_path = 'users_data.json'
users_data = load_user_data_from_json(json_file_path)

if users_data:
    print("User data loaded successfully.")
    print(users_data)
else:
    print("Failed to load user data.")

User data loaded successfully.
{'user_id': 123, 'user_name': 'Anas'}


In [33]:
# Example usage
config_file_path = 'config.toml'
connection_string, event_hub_name = read_config_from_toml(config_file_path)

In [34]:
send_events_to_event_hub(connection_string, event_hub_name, users_data)

Event data sent to Event Hub
